In [12]:
!pip install nltk
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords #Word Stop
from nltk.tokenize import word_tokenize #Tokenization & Word Stop
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation + '``'+ '`'+ ''+ ',' + '/')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rpierson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
#Define a Preprocessing subpipeline
class PreprocessingPineline:
    def __init__(self, stop_words, punctuation):
        self.punctuation = punctuation
        self.stop_words = stop_words
        self.data = None
        self.tokens = None
    def data_to_tokens(self, data):
        self.data = data
        self.tokens = self.data.astype(str).apply(word_tokenize)
        self.tokens = [[word for word in tokens if word.lower() not in stop_words and word not in punctuation] for tokens in self.tokens]
        return self.tokens
    
preprocess = PreprocessingPineline(stop_words, punctuation)

In [6]:
#Define a Topic Modeling subPipeline
class LDATopicModelPipeline:
    def __init__(self, lda_model_path, vectorizer_path):
        self.lda_model_path = lda_model_path
        self.vectorizer_path = vectorizer_path
        self.lda = None
        self.vectorizer = None
        
    def load_model(self):
        self.lda = joblib.load(self.lda_model_path)
        self.vectorizer = joblib.load(self.vectorizer_path)
        
    def topic_distributions(self, new_documents):
        if self.lda is None or self.vectorizer is None:
            self.load_model()
            
        docs_new = self.vectorizer.transform(new_documents)
        topic_distributions = self.lda.transform(docs_new)
        return topic_distributions

lda_model_path = '/home/rpierson/PiersonREU/extracted/lda.pkl'
vectorizer_path = '/home/rpierson/PiersonREU/extracted/vec.pkl'
#gives necessary files to process information
topic_model = LDATopicModelPipeline(lda_model_path, vectorizer_path)

topic_model.topic_distributions(docs)
#use through this syntax: topic_distributions = topic_model.transform(new_documents)

In [ ]:
#Define an Text Classification subpipeline
class TextClassificationNaiveBayes:

In [ ]:
#Using the equations for accuracy information
class AccuracyAssessment:
    def __init__():
    def precision():
    def recall():
    def fmeasure():
        
    

In [ ]:
def custom_pipeline(data):
    #PreProcessing
    #Topic Modeling (Insert Trained Model Here, Save as a .pth)
    #Text Classification Per Topic (Insert Trained Model Here, Save as a .pth)
    #Accuracy and Evaluation